In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import matplotlib.pyplot as plt
import logging

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Função para calcular o MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Carregar os dados
logging.info("Carregando dados...")
data = pd.read_excel("C:\\Users\\JoãoEmileXimenesMuri\\Documents\\pessoal\\TCC\\Bases TCC\\Dados_TCC_2014_2024_Normalizados_TabUnica.xlsx")

# Preparar os dados
logging.info("Preparando dados...")
data['IPCA_Next'] = data['Valor IPCA'].shift(-1)
data = data.dropna()
X = data.drop(columns=['Data Referência', 'Valor IPCA', 'IPCA_Next'])
y = data['IPCA_Next']

# Divisão treino/teste (80/20, mantendo ordem temporal)
train_size = int(len(data) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]
logging.info(f"Tamanho do conjunto de treino: {len(X_train)} observações")
logging.info(f"Tamanho do conjunto de teste: {len(X_test)} observações")

# Definir modelos e grades de hiperparâmetros expandidas
models = {
    'Ridge Regression': Ridge(),
    'SVR': SVR(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42)
}

param_grids = {
    'Ridge Regression': {
        'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]  # Ampliado
    },
    'SVR': {
        'C': [0.01, 0.1, 1.0, 10.0, 100.0],  # Ampliado
        'epsilon': [0.001, 0.01, 0.1, 0.2, 0.5],
        'kernel': ['rbf', 'linear', 'poly'],
        'degree': [2, 3]  # Para kernel poly
    },
    'Decision Tree': {
        'max_depth': [5, 6, 7, 8, 9],  # Foco em torno de 7
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 3]
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'max_depth': [2, 3, 4, 5],
        'min_samples_split': [2, 5, 10]
    },
    'Random Forest': {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [4, 5, 6, 7, 8],  # Foco em torno de 5
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 3]
    }
}

# Configurar validação cruzada temporal
tscv = TimeSeriesSplit(n_splits=5)

# Treinar e avaliar modelos com GridSearchCV
results = []
predictions = {}
best_params = {}

for name, model in models.items():
    logging.info(f"Otimizando {name}...")
    try:
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grids[name],
            cv=tscv,
            scoring='neg_mean_squared_error',
            n_jobs=1  # Sem paralelismo para evitar TerminatedWorkerError
        )
        grid_search.fit(X_train, y_train)
        
        # Armazenar melhores parâmetros
        best_params[name] = grid_search.best_params_
        logging.info(f"Melhores parâmetros para {name}: {best_params[name]}")
        
        # Treinar modelo com melhores parâmetros
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        
        # Calcular métricas
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Armazenar resultados
        results.append({
            'Model': name,
            'RMSE': rmse,
            'MAE': mae,
            'MAPE (%)': mape,
            'R2': r2
        })
        
        # Armazenar previsões
        predictions[name] = y_pred
    except Exception as e:
        logging.error(f"Erro ao otimizar {name}: {str(e)}")
        continue

2025-05-05 02:13:24,580 - INFO - Carregando dados...
2025-05-05 02:13:25,373 - INFO - Preparando dados...
2025-05-05 02:13:25,391 - INFO - Tamanho do conjunto de treino: 87 observações
2025-05-05 02:13:25,392 - INFO - Tamanho do conjunto de teste: 22 observações
2025-05-05 02:13:25,393 - INFO - Otimizando Ridge Regression...
2025-05-05 02:13:25,598 - INFO - Melhores parâmetros para Ridge Regression: {'alpha': 0.1}
2025-05-05 02:13:25,604 - INFO - Otimizando SVR...
2025-05-05 02:13:29,134 - INFO - Melhores parâmetros para SVR: {'C': 0.1, 'degree': 2, 'epsilon': 0.001, 'kernel': 'poly'}
2025-05-05 02:13:29,139 - INFO - Otimizando Decision Tree...
2025-05-05 02:13:30,370 - INFO - Melhores parâmetros para Decision Tree: {'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 2}
2025-05-05 02:13:30,376 - INFO - Otimizando Gradient Boosting...
2025-05-05 02:16:02,802 - INFO - Melhores parâmetros para Gradient Boosting: {'learning_rate': 0.2, 'max_depth': 3, 'min_samples_split': 5, 'n_es

In [2]:
# Exibir melhores parâmetros
print("\nMelhores Hiperparâmetros:")
for name, params in best_params.items():
    print(f"{name}: {params}")



Melhores Hiperparâmetros:
Ridge Regression: {'alpha': 0.1}
SVR: {'C': 0.1, 'degree': 2, 'epsilon': 0.001, 'kernel': 'poly'}
Decision Tree: {'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 2}
Gradient Boosting: {'learning_rate': 0.2, 'max_depth': 3, 'min_samples_split': 5, 'n_estimators': 50}
Random Forest: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 300}


In [3]:
# Criar DataFrame com resultados
results_df = pd.DataFrame(results)
print("\nResultados dos Modelos (com Hiperparâmetros Otimizados):")
print(results_df)


Resultados dos Modelos (com Hiperparâmetros Otimizados):
               Model      RMSE       MAE   MAPE (%)        R2
0   Ridge Regression  0.254381  0.232763  71.867596  0.063340
1                SVR  0.215762  0.180541  51.806877  0.326150
2      Decision Tree  0.117584  0.084284  21.679909  0.799871
3  Gradient Boosting  0.152836  0.124825  31.870249  0.661884
4      Random Forest  0.149297  0.124967  32.492663  0.677361


In [4]:
# Identificar o melhor modelo (menor RMSE)
if not results_df.empty:
    best_model = results_df.loc[results_df['RMSE'].idxmin()]
    print("\nMelhor Modelo (baseado no RMSE):")
    print(best_model)
else:
    print("\nNenhum modelo foi otimizado com sucesso.")


Melhor Modelo (baseado no RMSE):
Model       Decision Tree
RMSE             0.117584
MAE              0.084284
MAPE (%)        21.679909
R2               0.799871
Name: 2, dtype: object
